In [9]:
import pandas as pd
import sqlite3
import os

# 1. Setup Connection to a new SQLite Database
db_path = 'olist_store.db'
conn = sqlite3.connect(db_path)
print(f"Connected to database: {db_path}")

# 2. List of files to load (Update paths if your files are elsewhere)
# Key files needed for our analysis
files = {
    'orders': 'data/olist_orders_dataset.csv',
    'items': 'data/olist_order_items_dataset.csv',
    'products': 'data/olist_products_dataset.csv',
    'payments': 'data/olist_order_payments_dataset.csv',
    'customers': 'data/olist_customers_dataset.csv'
}

# 3. Load each CSV into a SQL Table
for table_name, file_path in files.items():
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path)
            
            # Fix Date Columns (Crucial for Time Series Analysis)
            if 'order_purchase_timestamp' in df.columns:
                df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
            
            # Write to SQL
            df.to_sql(table_name, conn, if_exists='replace', index=False)
            print(f"✅ Loaded {table_name} ({len(df)} rows)")
        else:
            print(f"❌ File not found: {file_path}")
    except Exception as e:
        print(f"⚠️ Error loading {table_name}: {e}")

print("\nDatabase setup complete!")
conn.close()


Connected to database: olist_store.db
✅ Loaded orders (99441 rows)
✅ Loaded items (112650 rows)
✅ Loaded products (32951 rows)
✅ Loaded payments (103886 rows)
✅ Loaded customers (99441 rows)

Database setup complete!


In [10]:
import pandas as pd
import sqlite3

# 1. Connect to your database
conn = sqlite3.connect('olist_store.db')

# 2. The Complex SQL Query (This is what you explain in interviews!)
# logic: Joins Orders -> Items -> Products -> Customers -> Payments
sql_query = """
SELECT 
    o.order_id,
    o.order_purchase_timestamp as order_date,
    o.order_status,
    
    -- Calculate Delivery Time (Actual vs Estimated)
    julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp) as delivery_days,
    julianday(o.order_estimated_delivery_date) - julianday(o.order_delivered_customer_date) as delivery_performance_days, -- Positive = Early, Negative = Late
    
    -- Customer Info
    c.customer_city,
    c.customer_state,
    
    -- Product Info
    p.product_category_name,
    
    -- Financials
    i.price as product_price,
    i.freight_value,
    pay.payment_type,
    pay.payment_installments,
    pay.payment_value as total_order_value

FROM orders o
JOIN items i ON o.order_id = i.order_id
JOIN products p ON i.product_id = p.product_id
JOIN customers c ON o.customer_id = c.customer_id
LEFT JOIN payments pay ON o.order_id = pay.order_id

WHERE o.order_status = 'delivered'
"""

print("Executing SQL Query... this might take a moment...")
df_master = pd.read_sql_query(sql_query, conn)

# 3. Quick Data Clean in Pandas (Easier than SQL for date formatting)
df_master['order_date'] = pd.to_datetime(df_master['order_date'])
df_master['year_month'] = df_master['order_date'].dt.to_period('M')

# 4. Save to CSV for Power BI
output_file = 'master_sales_data.csv'
df_master.to_csv(output_file, index=False)

print(f"✅ Success! Data extracted to '{output_file}' with {len(df_master)} rows.")
print("You can now import this file into Power BI.")
conn.close()


Executing SQL Query... this might take a moment...
✅ Success! Data extracted to 'master_sales_data.csv' with 115038 rows.
You can now import this file into Power BI.


In [12]:
import os
print("Your current working directory is:")
print(os.getcwd())

# Check if file exists there
file_name = 'master_sales_data.csv'
if os.path.exists(file_name):
    print(f"\nFOUND IT! It is here:\n{os.path.abspath(file_name)}")
else:
    print(f"\nStill lost. The file '{file_name}' is not in the current folder.")


Your current working directory is:
C:\Users\kawaa

FOUND IT! It is here:
C:\Users\kawaa\master_sales_data.csv
